In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
#read data
df_training = pd.read_csv("training_data.csv")
df_testing = pd.read_csv("test_data.csv")

/var/folders/t_/t17z84q161x8dcgl_1nslzxr0000gn/T/ipykernel_3179/1682910645.py:2: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_training = pd.read_csv("training_data.csv")


In [3]:
#replace null in winner_ratio as -1
df_testing['winner_ratio'] = -1
df_testing

,general_sector,city,zip_code,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,...,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,Food & Beverage,KILL DEVIL HILLS,27948.0,Restaurants & drinking establishments,NC,Individual,-1,6,6,0,...,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,1.000000,0.0,0.0
1,Uncoded,CHICAGO,60659.0,Uncoded,NJ,Individual,-1,2,0,2,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0
2,Lawyers & Lobbyists,ANKENY,50023.0,Lobbyists & public relations,IA,Individual,-1,2,0,2,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.0
3,Retired,MIAMI,33182.0,Retired,FL,Individual,-1,2,0,2,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.0
4,Uncoded,RIVERSIDE,8075.0,Uncoded,NJ,Non-Individual,-1,2,0,2,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115227,Lawyers & Lobbyists,RALEIGH,27612.0,Attorneys & law firms,US,Individual,-1,2,2,0,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.0,0.0
115228,Uncoded,ELEPHANT BUTTE,87935.0,Uncoded,NM,Individual,-1,2,0,2,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0
115229,Uncoded,BOCA RATON,33496.0,Uncoded,NJ,Individual,-1,2,0,2,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0
115230,Trucking,HARTFORD,6105.0,Trucking,CT,Non-Individual,-1,135,95,40,...,0.019563,0.525758,0.040404,0.656566,0.828283,0.131313,0.343434,0.417607,0.0,0.0


In [4]:
#encode nominal data to numeric 
df_training_number = df_training.copy()
le = LabelEncoder()
Nominal = ['general_sector','city','specific_sector','state','contributor_type']
for i in Nominal:    
    label = le.fit_transform(df_training[i])
    df_training_number[i+'_Label'] = label


In [5]:
#create mapping dataframe for encoding nominal data to numeric in testing

#mapping dataframe for column, general_sector
Map_general_sector_df = pd.DataFrame(columns=['general_sector','general_sector_Label'])
Map_general_sector_df['general_sector'] = df_training_number['general_sector']
Map_general_sector_df['general_sector_Label'] = df_training_number['general_sector_Label']
Map_general_sector_df = Map_general_sector_df.drop_duplicates()

#mapping dataframe for column, city
Map_city_df = pd.DataFrame(columns=['city','city_Label'])
Map_city_df['city'] = df_training_number['city']
Map_city_df['city_Label'] = df_training_number['city_Label']
Map_city_df = Map_city_df.drop_duplicates()

#mapping dataframe for column, specific_sector
Map_specific_sector_df = pd.DataFrame(columns= ['specific_sector','specific_sector_Label'])
Map_specific_sector_df['specific_sector'] = df_training_number['specific_sector']
Map_specific_sector_df['specific_sector_Label'] = df_training_number['specific_sector_Label']
Map_specific_sector_df = Map_specific_sector_df.drop_duplicates()

#mapping dataframe for column, state
Map_state_df = pd.DataFrame(columns=['state','state_Label'])
Map_state_df['state'] = df_training_number['state']
Map_state_df['state_Label'] = df_training_number['state_Label']
Map_state_df = Map_state_df.drop_duplicates()

#mapping dataframe for column, contributor_type
Map_contributor_type_df = pd.DataFrame(columns=['contributor_type','contributor_type_Label'])
Map_contributor_type_df['contributor_type'] = df_training_number['contributor_type']
Map_contributor_type_df['contributor_type_Label'] = df_training_number['contributor_type_Label']
Map_contributor_type_df = Map_contributor_type_df.drop_duplicates()

In [6]:
#replace the original nominal attribute to the label ones
df_training_number = df_training_number.drop(columns = ['general_sector','city','specific_sector','state','contributor_type'])
df_training_number = df_training_number.rename(columns={"general_sector_Label":"general_sector","city_Label":"city","specific_sector_Label":"specific_sector","state_Label":"state","contributor_type_Label":"contributor_type"})

In [7]:
#transform nominal data to numeric 
df_testing_number = df_testing.copy()
df_testing_number =  df_testing_number.merge(Map_general_sector_df[['general_sector','general_sector_Label']], how ='left', left_on = ['general_sector'], right_on =['general_sector'])
df_testing_number =  df_testing_number.merge(Map_city_df[['city','city_Label']], how ='left', left_on = ['city'], right_on =['city'])
df_testing_number =  df_testing_number.merge(Map_specific_sector_df[['specific_sector','specific_sector_Label']], how ='left', left_on = ['specific_sector'], right_on =['specific_sector'])
df_testing_number =  df_testing_number.merge(Map_state_df[['state','state_Label']], how ='left', left_on = ['state'], right_on =['state'])
df_testing_number =  df_testing_number.merge(Map_contributor_type_df[['contributor_type','contributor_type_Label']], how ='left', left_on = ['contributor_type'], right_on =['contributor_type'])
df_testing_number = df_testing_number.drop(columns = ['general_sector','city','specific_sector','state','contributor_type'])

df_testing_number = df_testing_number.rename(columns={"general_sector_Label":"general_sector","city_Label":"city","specific_sector_Label":"specific_sector","state_Label":"state","contributor_type_Label":"contributor_type"})



In [8]:
df_testing_number

,zip_code,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,general_sector,city,specific_sector,state,contributor_type
0,27948.0,-1,6,6,0,10,10,0,0,4,...,0.000000,0.000000,1.000000,0.0,0.0,45,4036.0,341.0,24.0,0
1,60659.0,-1,2,0,2,2,0,2,0,2,...,0.000000,1.000000,0.000000,0.0,0.0,128,1413.0,395.0,28.0,0
2,50023.0,-1,2,0,2,2,0,2,0,2,...,1.000000,1.000000,0.000000,0.0,0.0,66,204.0,215.0,11.0,0
3,33182.0,-1,2,0,2,6,0,6,0,2,...,1.000000,1.000000,0.000000,0.0,0.0,110,5009.0,344.0,8.0,0
4,8075.0,-1,2,0,2,2,0,2,0,2,...,0.000000,1.000000,1.000000,0.0,0.0,128,6794.0,395.0,28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115227,27612.0,-1,2,2,0,2,2,0,0,2,...,0.000000,0.000000,0.000000,1.0,0.0,66,6603.0,27.0,40.0,0
115228,87935.0,-1,2,0,2,2,0,2,0,2,...,0.000000,1.000000,0.000000,0.0,0.0,128,2352.0,395.0,29.0,0
115229,33496.0,-1,2,0,2,2,0,2,0,2,...,0.000000,1.000000,0.000000,0.0,0.0,128,754.0,395.0,28.0,0
115230,6105.0,-1,135,95,40,152,105,47,4,99,...,0.131313,0.343434,0.417607,0.0,0.0,127,3369.0,392.0,5.0,1


In [9]:
#Transform datatype to float
for i in range(len(df_training_number['governor_contribution_ratio'])):
    if type(df_training_number['governor_contribution_ratio'][i]) != float and df_training_number['governor_contribution_ratio'][i] !="#NAME?":
        df_training_number['governor_contribution_ratio'][i] = float(df_training_number['governor_contribution_ratio'][i])

# set #NAME? to null
df_training_number['governor_contribution_ratio'][149673] = None
        
for i in range(len(df_training_number['house_and_assembly_contribution_ratio'])):
    if type(df_training_number['house_and_assembly_contribution_ratio'][i]) != float and df_training_number['house_and_assembly_contribution_ratio'][i] !="#NAME?":
        df_training_number['house_and_assembly_contribution_ratio'][i] = float(df_training_number['house_and_assembly_contribution_ratio'][i])
    # set inf to null
    if df_training_number["house_and_assembly_contribution_ratio"][i] == float('inf'):
        df_training_number["house_and_assembly_contribution_ratio"][i] = None
        
for i in range(len(df_training_number['senate_contribution_ratio'])):
    if type(df_training_number['senate_contribution_ratio'][i]) != float and df_training_number['senate_contribution_ratio'][i] !="#NAME?":
        df_training_number['senate_contribution_ratio'][i] = float(df_training_number['senate_contribution_ratio'][i])
# set #NAME? to null
df_training_number['senate_contribution_ratio'][129136] = None

for i in range(len(df_training_number['us_house_contribution_ratio'])):
    if type(df_training_number['us_house_contribution_ratio'][i]) != float and df_training_number['us_house_contribution_ratio'][i] !="#NAME?":
        df_training_number['us_house_contribution_ratio'][i] = float(df_training_number['us_house_contribution_ratio'][i])
     # set inf to null
    if df_training_number["us_house_contribution_ratio"][i] == float('inf'):
        df_training_number["us_house_contribution_ratio"][i] = None
        
for i in range(len(df_training_number['us_senate_contribution_ratio'])):
    if type(df_training_number['us_senate_contribution_ratio'][i]) != float and df_training_number['us_senate_contribution_ratio'][i] !="#NAME?":
        df_training_number['us_senate_contribution_ratio'][i] = float(df_training_number['us_senate_contribution_ratio'][i])  
# set #NAME? to null
df_training_number['us_senate_contribution_ratio'][126904] = None

/var/folders/t_/t17z84q161x8dcgl_1nslzxr0000gn/T/ipykernel_3179/3500085911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training_number['governor_contribution_ratio'][i] = float(df_training_number['governor_contribution_ratio'][i])
/var/folders/t_/t17z84q161x8dcgl_1nslzxr0000gn/T/ipykernel_3179/3500085911.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training_number['governor_contribution_ratio'][149673] = None
/var/folders/t_/t17z84q161x8dcgl_1nslzxr0000gn/T/ipykernel_3179/3500085911.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

In [10]:
#Training data
#set mising value as 0
for i in Nominal:
    df_training_number[i] = df_training_number[i].fillna(0)
df_training_number['zip_code'] = df_training_number['zip_code'].fillna(0)
#imputation the missing value with mean
df_training_number["governor_contribution_ratio"] = df_training_number["governor_contribution_ratio"].fillna(df_training_number["governor_contribution_ratio"].mean())
df_training_number["house_and_assembly_contribution_ratio"] = df_training_number["house_and_assembly_contribution_ratio"].fillna(df_training_number["house_and_assembly_contribution_ratio"].mean())
df_training_number["senate_contribution_ratio"] = df_training_number["senate_contribution_ratio"].fillna(df_training_number["senate_contribution_ratio"].mean())
df_training_number["us_house_contribution_ratio"] = df_training_number["us_house_contribution_ratio"].fillna(df_training_number["us_house_contribution_ratio"].mean())
df_training_number["us_senate_contribution_ratio"] = df_training_number["us_senate_contribution_ratio"].fillna(df_training_number["us_senate_contribution_ratio"].mean())


#Testing data
#set mising value as 0
for i in Nominal:
    df_testing_number[i] = df_testing_number[i].fillna(0)
df_testing_number['zip_code'] = df_testing_number['zip_code'].fillna(0)
#imputation the missing value with mean
df_testing_number["governor_contribution_ratio"] = df_testing_number["governor_contribution_ratio"].fillna(df_testing_number["governor_contribution_ratio"].mean())
df_testing_number["house_and_assembly_contribution_ratio"] = df_testing_number["house_and_assembly_contribution_ratio"].fillna(df_testing_number["house_and_assembly_contribution_ratio"].mean())
df_testing_number["senate_contribution_ratio"] = df_testing_number["senate_contribution_ratio"].fillna(df_testing_number["senate_contribution_ratio"].mean())
df_testing_number["us_house_contribution_ratio"] = df_testing_number["us_house_contribution_ratio"].fillna(df_testing_number["us_house_contribution_ratio"].mean())
df_testing_number["us_senate_contribution_ratio"] = df_testing_number["us_senate_contribution_ratio"].fillna(df_testing_number["us_senate_contribution_ratio"].mean())


In [11]:
#split training and testing data
X_train = df_training_number.drop(columns = ["winner_ratio"])
y_train = df_training_number["winner_ratio"]
X_test = df_testing_number.drop(columns = ["winner_ratio"])

In [12]:
#Model-random forest

In [13]:
#depth:25 random_state = 0
regr = RandomForestRegressor(max_depth=35, random_state=0)
RF_model = regr.fit(X_train, y_train)
RF_y_pred = RF_model.predict(X_test)

In [14]:
RF_solution = pd.DataFrame(RF_y_pred,columns=['winner_ratio'])

In [15]:
RF_solution

,winner_ratio
0,0.831667
1,1.000000
2,0.596667
3,0.561814
4,0.755000
...,...
115227,0.745000
115228,0.820000
115229,1.000000
115230,0.863326


In [16]:
#export csv
RF_solution.to_csv("RF_solution.csv",index_label = 'index')